In [1]:
import numpy
from math import pi
%matplotlib inline

In [2]:
def plot_3D(x, y, p):

    fig = pyplot.figure(figsize=(11,7), dpi=100)
    ax = fig.gca(projection='3d')
    X,Y = numpy.meshgrid(x,y)
    surf = ax.plot_surface(X,Y,p[:], rstride=1, cstride=1, cmap=cm.viridis,
            linewidth=0, antialiased=False)


    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.view_init(30,45)

In [3]:
def L2_rel_error(p, pn):
    return numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))

In [4]:
def p_analytical(X,Y):

    return numpy.sin(X*pi/5)*numpy.cos(Y*pi/5)

In [5]:
def poisson_2d(p, b, dx, dy, l2_target):

    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    while l2_norm > l2_target:

        pd = p.copy()

        p[1:-1,1:-1] = 1/(2*(dx**2 + dy**2)) * \
                        ((pd[1:-1,2:]+pd[1:-1,:-2])*dy**2 +\
                        (pd[2:,1:-1] + pd[:-2,1:-1])*dx**2 -\
                         b[1:-1,1:-1]*dx**2*dy**2)
    
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of Jacobi iterations: {0:d}'.format(iterations))
    return p, l2_conv    

In [7]:
# Parameters
nx = 101
ny = 101
xmin = 0
xmax = 1
ymin = -0.5
ymax = 0.5

l2_target = 1e-10

# Spacing
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

# Mesh
x  = numpy.linspace(xmin,xmax,nx)
y  = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

# Source
L = xmax-xmin
b = -2*(pi/L)**2*numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)

# Initialization
p_i  = numpy.zeros((ny,nx))

# Analytical solution
pan = p_analytical(X,Y)

In [8]:
def steepest_descent_2d(p, b, dx, dy, l2_target):

    ny, nx = p.shape
    r  = numpy.zeros((ny,nx)) # residual
    Ar  = numpy.zeros((ny,nx)) # to store result of matrix multiplication
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Iterations
    while l2_norm > l2_target:

        pd = p.copy()
        
        r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:, 1:-1] - pd[:-2, 1:-1]
        
        Ar[1:-1,1:-1] = -4*r[1:-1,1:-1]+r[1:-1,2:]+r[1:-1,:-2]+\
            r[2:, 1:-1] + r[:-2, 1:-1]

        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma

        p = pd + alpha*r
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of SD iterations: {0:d}'.format(iterations))
    return p, l2_conv     

In [9]:
p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p, pan)

Number of SD iterations: 2


1.0173874060072625

In [10]:
def conjugate_gradient_2d(p, b, dx, dy, l2_target):

    ny, nx = p.shape
    r  = numpy.zeros((ny,nx)) # residual
    Ad  = numpy.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual
    
    r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = numpy.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma

        p = pk + alpha*dk
        r = rk- alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv  

In [11]:
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

Number of CG iterations: 2


1.0173874060072625

In [12]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31227


In [13]:
b = (numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)+
     numpy.sin(6*pi*X/L)*numpy.sin(6*pi*Y/L))

In [14]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31226
Number of SD iterations: 28179
Number of CG iterations: 3
